# MIDS W206 Assignment 1
Student: Kuan Lin

* <b>HW1.0.0. Define big data. Provide an example of a big data problem in your domain of expertise</b>

to do....

* <b>HW1.0.1.In 500 words (English or pseudo code or a combination) describe how to estimate the bias, the variance, the irreduciable error for a test dataset T when using polynomial regression models of degree 1, 2,3, 4,5 are considered. How would you select a model?</b>

to do...

<b>spam filter section</b>

In [2]:
%%writefile pNaiveBayes.sh
## pNaiveBayes.sh
## Author: Jake Ryland Williams
## Usage: pNaiveBayes.sh m wordlist
## Input:
##       m = number of processes (maps), e.g., 4
##       wordlist = a space-separated list of words in quotes, e.g., "the and of"
##
## Instructions: Read this script and its comments closely.
##               Do your best to understand the purpose of each command,
##               and focus on how arguments are supplied to mapper.py/reducer.py,
##               as this will determine how the python scripts take input.
##               When you are comfortable with the unix code below,
##               answer the questions on the LMS for HW1 about the starter code.

## collect user input
m=$1 ## the number of parallel processes (maps) to run
wordlist=$2 ## if set to "*", then all words are used

## a test set data of 100 messages
data="enronemail_1h.txt" 

## the full set of data (33746 messages)
# data="enronemail.txt" 

## 'wc' determines the number of lines in the data
## 'perl -pe' regex strips the piped wc output to a number
linesindata=`wc -l $data | perl -pe 's/^.*?(\d+).*?$/$1/'`

## determine the lines per chunk for the desired number of processes
linesinchunk=`echo "$linesindata/$m+1" | bc`

## split the original file into chunks by line
split -l $linesinchunk $data $data.chunk.

## assign python mappers (mapper.py) to the chunks of data
## and emit their output to temporary files
for datachunk in $data.chunk.*; do
    ## feed word list to the python mapper here and redirect STDOUT to a temporary file on disk
    ####
    ####
    ./mapper.py $datachunk "$wordlist" > $datachunk.counts &
    ####
    ####
done
## wait for the mappers to finish their work
wait

## 'ls' makes a list of the temporary count files
## 'perl -pe' regex replaces line breaks with spaces
countfiles=`\ls $data.chunk.*.counts | perl -pe 's/\n/ /'`

## feed the list of countfiles to the python reducer and redirect STDOUT to disk
####
####
./reducer.py $countfiles > $data.output
####
####

## clean up the data chunks and temporary count files
\rm $data.chunk.*

Writing pNaiveBayes.sh


In [3]:
!chmod a+x pNaiveBayes.sh

* <b>HW1.1. Read through the provided control script (pNaiveBayes.sh)
   and all of its comments. When you are comfortable with their
   purpose and function, respond to the remaining homework questions below.</b>

done

* <b>HW1.2. Provide a mapper/reducer pair that, when executed by pNaiveBayes.sh
   will determine the number of occurrences of a single, user-specified word. Examine the word “assistance” and report your results.</b>

In [6]:
%%writefile mapper.py
#!/usr/bin/python
## mapper.py
## Author: Kuan Lin
## Description: mapper code for HW1.2

import sys
import re
import string

## collect user input
filename = sys.argv[1]
if sys.argv[2] == "*":
	useAllWords = True
else:
	useAllWords = False
	findwords = re.split(" ",sys.argv[2].lower())

word_counts = {}

for line in open(filename, 'r'):
	line = line.strip().lower()
	if line == "": continue
	lineArr = line.split('\t')
	# check data integrity.  each valid line should have 4 fields, with second field equals either 0 or 1
	if len(lineArr) != 4 or (lineArr[1] != "0" and lineArr[1] != "1"): continue
	
	spam_flag = lineArr[1]
	subject_body = lineArr[2] + ' ' + lineArr[3]
	# remove punctuations
	subject_body = subject_body.translate(string.maketrans("",""), string.punctuation)
	
	if useAllWords:
		findwords = set([word.strip() for word in subject_body.split(' ') if word.strip() != ''])
	
	for word in findwords:
		try:
			pattern = re.compile('[\s+]?(' + word + ')[\s+]?')
			if word in word_counts:
				word_counts[word] += len(re.findall(pattern, subject_body))
			else:
				word_counts[word] = len(re.findall(pattern, subject_body))
		except:
			if word in word_counts:
				word_counts[word] += subject_body.count(word)
			else:
				word_counts[word] = subject_body.count(word)
		
print '\n'.join([word + '\t' + str(word_counts[word]) for word in word_counts])

Writing mapper.py


In [7]:
%%writefile reducer.py
#!/usr/bin/python
## reducer.py
## Author: Kuan Lin
## Description: reducer code for HW1.2

import sys
import re

files = sys.argv[1:]
#print str(files)

word_counts = {}
for file in files:
	for line in open(file, 'r'):
		line = line.strip()
		if line == "": continue
		
		word = line.split('\t')[0]
		count = int(line.split('\t')[1])
		
		if word in word_counts:
			word_counts[word] += count
		else:
			word_counts[word] = count
			
print '\n'.join([word + '\t' + str(word_counts[word]) for word in word_counts])

Writing reducer.py


In [9]:
!chmod +x mapper.py; chmod +x reducer.py; ./pNaiveBayes.sh 4 assistance

In [10]:
!cat enronemail_1h.txt.output

assistance	9


2 email records were excluded due to bad formatting in the source data.  In the remaining valid email entries there are 9 instances of the word "assistance" in the email subject and body.

* <b>HW1.3. Provide a mapper/reducer pair that, when executed by pNaiveBayes.sh
   will classify the email messages by a single, user-specified word using the multinomial Naive Bayes Formulation. Examine the word “assistance” and report your results.</b>

In [11]:
%%writefile mapper.py
#!/usr/bin/python
## mapper.py
## Author: Kuan Lin
## Description: mapper code for HW1.3

import sys
import re
import string

## collect user input
filename = sys.argv[1]
if sys.argv[2] == "*":
	useAllWords = True
else:
	useAllWords = False
	findwords = re.split(" ",sys.argv[2].lower())

spam_word_counts = {}
ham_word_counts = {}
spam_total_words = 0
ham_total_words = 0

for line in open(filename, 'r'):
	line = line.strip().lower()
	if line == "": continue
	lineArr = line.split('\t')
	# check data integrity.  each valid line should have 4 fields, with second field equals either 0 or 1
	if len(lineArr) != 4 or (lineArr[1] != "0" and lineArr[1] != "1"): continue
	
	spam_flag = lineArr[1]
	subject_body = lineArr[2] + ' ' + lineArr[3]
	# remove punctuations
	subject_body = subject_body.translate(string.maketrans("",""), string.punctuation).strip()
	all_words = [word.strip() for word in subject_body.split(' ') if word.strip() != '']
	if useAllWords:
		findwords = set(all_words)
		
	if spam_flag == "1":
		spam_total_words += len(all_words)
	else:
		ham_total_words += len(all_words)
	
	for word in findwords:
		pattern = re.compile('[\s+]?(' + word + ')[\s+]?')
		if spam_flag == "1":
			try:
				if word in spam_word_counts: spam_word_counts[word] += len(re.findall(pattern, subject_body))
				else: spam_word_counts[word] = len(re.findall(pattern, subject_body))
			except:
				if word in spam_word_counts: spam_word_counts[word] += subject_body.count(word)
				else: spam_word_counts[word] = subject_body.count(word)
		else:
			try:
				if word in ham_word_counts: ham_word_counts[word] += len(re.findall(pattern, subject_body))
				else: ham_word_counts[word] = len(re.findall(pattern, subject_body))
			except:
				if word in ham_word_counts: ham_word_counts[word] += subject_body.count(word)
				else: ham_word_counts[word] = subject_body.count(word)
			
		
print '\n'.join([word + '\t' + str(spam_word_counts[word]) + '\t1' for word in spam_word_counts] + 
[word + '\t' + str(ham_word_counts[word]) + '\t0' for word in ham_word_counts] +
['class_total_words\t' + str(spam_total_words) + '\t1', 'class_total_words\t' + str(ham_total_words) + '\t0'])

Overwriting mapper.py


In [12]:
%%writefile reducer.py
#!/usr/bin/python
## reducer.py
## Author: Kuan Lin
## Description: reducer code for HW1.3

import sys
import re
import math
import string

files = sys.argv[1:]
data="enronemail_1h.txt"

spam_word_counts = {}
ham_word_counts = {}
spam_total_words = 0
ham_total_words = 0

# assume equal prior for spam and ham
spam_prior = 0.44
ham_prior = 0.56

def spamWordProb(word):
	count = 0
	if word in spam_word_counts: count = spam_word_counts[word]
	# apply laplace smoothing
	return float(count+1)/float(spam_total_words+len(spam_word_counts))
	
def hamWordProb(word):
	count = 0
	if word in ham_word_counts: count = ham_word_counts[word]
	# apply laplace smoothing
	return float(count+1)/float(ham_total_words+len(ham_word_counts))
	
def classifyWithWordList(document, word_list):
	spam_logprob = math.log(spam_prior)
	ham_logprob = math.log(ham_prior)
	for word in word_list:
		pattern = re.compile('[\s+]?(' + word + ')[\s+]?')
		word_count = len(re.findall(pattern, document))
		spam_logprob += math.log(spamWordProb(word))*word_count
		ham_logprob += math.log(hamWordProb(word))*word_count
		
	if spam_logprob > ham_logprob:
		return 1
	else:
		return 0

for file in files:
	for line in open(file, 'r'):
		line = line.strip()
		if line == "": continue
		
		lineArr = line.split('\t')
		word = lineArr[0]
		count = int(lineArr[1])
		spam_flag = int(lineArr[2])
		
		if spam_flag == 1:
			if word == "class_total_words":
				spam_total_words += count
				continue
			if word in spam_word_counts:
				spam_word_counts[word] += count
			else:
				spam_word_counts[word] = count
		else:
			if word == "class_total_words":
				ham_total_words += count
				continue
			if word in ham_word_counts:
				ham_word_counts[word] += count
			else:
				ham_word_counts[word] = count
			
# perform single-word classification
key_word = spam_word_counts.keys()[0]
total_msg_classified = 0
total_msg_correct = 0
for line in open(data, 'r'):
	line = line.strip()
	lineArr = line.split('\t')
	# check data integrity.  each valid line should have 4 fields, with second field equals either 0 or 1
	if len(lineArr) != 4 or (lineArr[1] != "0" and lineArr[1] != "1"): continue
	msg_id = lineArr[0]
	spam_flag = lineArr[1]
	subject_body = lineArr[2] + ' ' + lineArr[3]
	# remove punctuations
	subject_body = subject_body.translate(string.maketrans("",""), string.punctuation).strip()
	spam_flag_predicted = str(classifyWithWordList(subject_body, [key_word]))
	total_msg_classified += 1
	if spam_flag_predicted == spam_flag: total_msg_correct += 1
	print '\t'.join([msg_id, spam_flag, spam_flag_predicted])
print "-----------------------------------------------------"
print "total document classified: " + str(total_msg_classified)
print "total document correctly classified: " + str(total_msg_correct)
print "accuracy: %.4f"%(float(total_msg_correct)/float(total_msg_classified))

Overwriting reducer.py


In [13]:
!chmod +x mapper.py; chmod +x reducer.py; ./pNaiveBayes.sh 4 assistance

In [14]:
!tail -3 enronemail_1h.txt.output

total document classified: 98
total document correctly classified: 58
accuracy: 0.5918


again, two documents were excluded due to bad formating in the source data

* <b>HW1.4. Provide a mapper/reducer pair that, when executed by pNaiveBayes.sh
   will classify the email messages by a list of one or more user-specified words. Examine the words “assistance”, “valium”, and “enlargementWithATypo” and report your results</b>

In [15]:
%%writefile mapper.py
#!/usr/bin/python
## mapper.py
## Author: Kuan Lin
## Description: mapper code for HW1.4

import sys
import re
import string

## collect user input
filename = sys.argv[1]
if sys.argv[2] == "*":
	useAllWords = True
else:
	useAllWords = False
	findwords = re.split(" ",sys.argv[2].lower())

spam_word_counts = {}
ham_word_counts = {}
spam_total_words = 0
ham_total_words = 0

for line in open(filename, 'r'):
	line = line.strip().lower()
	if line == "": continue
	lineArr = line.split('\t')
	# check data integrity.  each valid line should have 4 fields, with second field equals either 0 or 1
	if len(lineArr) != 4 or (lineArr[1] != "0" and lineArr[1] != "1"): continue
	
	spam_flag = lineArr[1]
	subject_body = lineArr[2] + ' ' + lineArr[3]
	# remove punctuations
	subject_body = subject_body.translate(string.maketrans("",""), string.punctuation).strip().lower()
	all_words = [word.strip() for word in subject_body.split(' ') if word.strip() != '']
	if useAllWords:
		findwords = set(all_words)
		
	if spam_flag == "1":
		spam_total_words += len(all_words)
	else:
		ham_total_words += len(all_words)
	
	for word in findwords:
		pattern = re.compile('[\s+]?(' + word + ')[\s+]?')
		if spam_flag == "1":
			try:
				if word in spam_word_counts: spam_word_counts[word] += len(re.findall(pattern, subject_body))
				else: spam_word_counts[word] = len(re.findall(pattern, subject_body))
			except:
				if word in spam_word_counts: spam_word_counts[word] += subject_body.count(word)
				else: spam_word_counts[word] = subject_body.count(word)
		else:
			try:
				if word in ham_word_counts: ham_word_counts[word] += len(re.findall(pattern, subject_body))
				else: ham_word_counts[word] = len(re.findall(pattern, subject_body))
			except:
				if word in ham_word_counts: ham_word_counts[word] += subject_body.count(word)
				else: ham_word_counts[word] = subject_body.count(word)
			
		
print '\n'.join([word + '\t' + str(spam_word_counts[word]) + '\t1' for word in spam_word_counts] + 
[word + '\t' + str(ham_word_counts[word]) + '\t0' for word in ham_word_counts] +
['class_total_words\t' + str(spam_total_words) + '\t1', 'class_total_words\t' + str(ham_total_words) + '\t0'])

Overwriting mapper.py


In [16]:
%%writefile reducer.py
#!/usr/bin/python
## reducer.py
## Author: Kuan Lin
## Description: reducer code for HW1.4

import sys
import re
import math
import string

files = sys.argv[1:]
data="enronemail_1h.txt"

spam_word_counts = {}
ham_word_counts = {}
spam_total_words = 0
ham_total_words = 0

# assume equal prior for spam and ham
spam_prior = 0.44
ham_prior = 0.56

def spamWordProb(word):
	count = 0
	if word in spam_word_counts: count = spam_word_counts[word]
	# apply laplace smoothing
	return float(count+1)/float(spam_total_words+len(spam_word_counts))
	
def hamWordProb(word):
	count = 0
	if word in ham_word_counts: count = ham_word_counts[word]
	# apply laplace smoothing
	return float(count+1)/float(ham_total_words+len(ham_word_counts))
	
def classifyWithWordList(document, word_list):
	spam_logprob = math.log(spam_prior)
	ham_logprob = math.log(ham_prior)
	for word in word_list:
		pattern = re.compile('[\s+]?(' + word + ')[\s+]?')
		word_count = len(re.findall(pattern, document))
		spam_logprob += math.log(spamWordProb(word))*word_count
		ham_logprob += math.log(hamWordProb(word))*word_count
		
	if spam_logprob > ham_logprob:
		return 1
	else:
		return 0

for file in files:
	for line in open(file, 'r'):
		line = line.strip()
		if line == "": continue
		
		lineArr = line.split('\t')
		word = lineArr[0]
		count = int(lineArr[1])
		spam_flag = int(lineArr[2])
		
		if spam_flag == 1:
			if word == "class_total_words":
				spam_total_words += count
				continue
			if word in spam_word_counts:
				spam_word_counts[word] += count
			else:
				spam_word_counts[word] = count
		else:
			if word == "class_total_words":
				ham_total_words += count
				continue
			if word in ham_word_counts:
				ham_word_counts[word] += count
			else:
				ham_word_counts[word] = count
			
# perform classification with a word-list
key_words = spam_word_counts.keys()
total_msg_classified = 0
total_msg_correct = 0
for line in open(data, 'r'):
	line = line.strip()
	lineArr = line.split('\t')
	# check data integrity.  each valid line should have 4 fields, with second field equals either 0 or 1
	if len(lineArr) != 4 or (lineArr[1] != "0" and lineArr[1] != "1"): continue
	msg_id = lineArr[0]
	spam_flag = lineArr[1]
	subject_body = lineArr[2] + ' ' + lineArr[3]
	# remove punctuations
	subject_body = subject_body.translate(string.maketrans("",""), string.punctuation).strip()
	spam_flag_predicted = str(classifyWithWordList(subject_body, key_words))
	total_msg_classified += 1
	if spam_flag_predicted == spam_flag: total_msg_correct += 1
	print '\t'.join([msg_id, spam_flag, spam_flag_predicted])
print "-----------------------------------------------------"
print "total document classified: " + str(total_msg_classified)
print "total document correctly classified: " + str(total_msg_correct)
print "accuracy: %.4f"%(float(total_msg_correct)/float(total_msg_classified))

Overwriting reducer.py


In [19]:
!chmod +x mapper.py; chmod +x reducer.py; ./pNaiveBayes.sh 4 "assistance valium enlargementWithATypo"

In [20]:
!tail -3 enronemail_1h.txt.output

total document classified: 98
total document correctly classified: 61
accuracy: 0.6224


* <b>HW1.5. Provide a mapper/reducer pair that, when executed by pNaiveBayes.sh
   will classify the email messages by all words present.</b>

In [22]:
%%writefile mapper.py
#!/usr/bin/python
## mapper.py
## Author: Kuan Lin
## Description: mapper code for HW1.5

import sys
import re
import string

## collect user input
filename = sys.argv[1]
if sys.argv[2] == "*":
	useAllWords = True
	findwords = None
else:
	useAllWords = False
	findwords = re.split(" ",sys.argv[2].lower())

spam_word_counts = {}
ham_word_counts = {}
spam_total_words = 0
ham_total_words = 0

def findWordCount(doc, word):
	count = 0
	for w in doc.split(' '):
		if w == word: count += 1
	return count

for line in open(filename, 'r'):
	line = line.strip().lower()
	if line == "": continue
	lineArr = line.split('\t')
	# check data integrity.  each valid line should have 4 fields, with second field equals either 0 or 1
	if len(lineArr) != 4 or (lineArr[1] != "0" and lineArr[1] != "1"): continue
	
	spam_flag = lineArr[1]
	subject_body = lineArr[2] + ' ' + lineArr[3]
	# remove punctuations
	subject_body = subject_body.translate(string.maketrans("",""), string.punctuation).strip().lower()
	all_words = [word.strip() for word in subject_body.split(' ') if word.strip() != '']
	#print str(all_words)
	if useAllWords:
		findwords = set([w for w in all_words])
	
	if spam_flag == "1":
		spam_total_words += len(all_words)
	else:
		ham_total_words += len(all_words)
		
	for word in findwords:
		#pattern = re.compile('[\s+]?(' + word + ')[\s+]?')
		if spam_flag == "1":
			if word in spam_word_counts: spam_word_counts[word] += findWordCount(subject_body, word)
			else: spam_word_counts[word] = findWordCount(subject_body, word)
		else:
			if word in ham_word_counts: ham_word_counts[word] += findWordCount(subject_body, word)
			else: ham_word_counts[word] = findWordCount(subject_body, word)
			
print '\n'.join([word + '\t' + str(spam_word_counts[word]) + '\t1' for word in spam_word_counts] + 
[word + '\t' + str(ham_word_counts[word]) + '\t0' for word in ham_word_counts] +
['class_total_words\t' + str(spam_total_words) + '\t1', 'class_total_words\t' + str(ham_total_words) + '\t0'])

Overwriting mapper.py


In [23]:
%%writefile reducer.py
#!/usr/bin/python
## reducer.py
## Author: Kuan Lin
## Description: reducer code for HW1.5

import sys
import re
import math
import string

files = sys.argv[1:]
data="enronemail_1h.txt"

spam_word_counts = {}
ham_word_counts = {}
spam_total_words = 0
ham_total_words = 0
unique_words = set()

# assume equal prior for spam and ham
spam_prior = 0.44
ham_prior = 0.56

def findWordCount(doc, word):
	count = 0
	for w in doc.split(' '):
		if w == word: count += 1
	return count

def spamWordProb(word):
	count = 0
	if word in spam_word_counts: count = spam_word_counts[word]
	# apply laplace smoothing
	return float(count+1)/float(spam_total_words+len(unique_words))
	
def hamWordProb(word):
	count = 0
	if word in ham_word_counts: count = ham_word_counts[word]
	# apply laplace smoothing
	return float(count+1)/float(ham_total_words+len(unique_words))
	
def classifyWithWordList(document, word_list):
	spam_logprob = math.log(spam_prior)
	ham_logprob = math.log(ham_prior)
	for word in word_list:
		#pattern = re.compile('[\s+]?(' + word + ')[\s+]?')
		word_count = findWordCount(document, word)
		spam_logprob += math.log(spamWordProb(word))*word_count
		ham_logprob += math.log(hamWordProb(word))*word_count
	
	#print "%.4f | %.4f" % (spam_logprob, ham_logprob)
	if spam_logprob > ham_logprob:
		return 1
	else:
		return 0

for file in files:
	for line in open(file, 'r'):
		line = line.strip()
		if line == "": continue
		
		lineArr = line.split('\t')
		word = lineArr[0]
		count = int(lineArr[1])
		spam_flag = int(lineArr[2])
		
		if spam_flag == 1:
			if word == "class_total_words":
				spam_total_words += count
				continue
			if word in spam_word_counts:
				spam_word_counts[word] += count
			else:
				spam_word_counts[word] = count
		else:
			if word == "class_total_words":
				ham_total_words += count
				continue
			if word in ham_word_counts:
				ham_word_counts[word] += count
			else:
				ham_word_counts[word] = count
			
# perform classification with all words
unique_words = set([k for k in spam_word_counts.keys()] + [k for k in ham_word_counts.keys()])
total_msg_classified = 0
total_msg_correct = 0
for line in open(data, 'r'):
	line = line.strip()
	lineArr = line.split('\t')
	# check data integrity.  each valid line should have 4 fields, with second field equals either 0 or 1
	if len(lineArr) != 4 or (lineArr[1] != "0" and lineArr[1] != "1"): continue
	msg_id = lineArr[0]
	spam_flag = lineArr[1]
	subject_body = lineArr[2] + ' ' + lineArr[3]
	# remove punctuations
	subject_body = subject_body.translate(string.maketrans("",""), string.punctuation).strip().lower()
	spam_flag_predicted = str(classifyWithWordList(subject_body, unique_words))
	total_msg_classified += 1
	if spam_flag_predicted == spam_flag: total_msg_correct += 1
	print '\t'.join([msg_id, spam_flag, spam_flag_predicted])
print "-----------------------------------------------------"
print "feature size: " + str(len(unique_words))
print "spam/ham total words: %s | %s" % (spam_total_words, ham_total_words)
print "total document classified: " + str(total_msg_classified)
print "total document correctly classified: " + str(total_msg_correct)
print "accuracy: %.4f"%(float(total_msg_correct)/float(total_msg_classified))

Overwriting reducer.py


In [24]:
!chmod +x mapper.py; chmod +x reducer.py; ./pNaiveBayes.sh 4 "*"

In [25]:
!tail -3 enronemail_1h.txt.output

total document classified: 98
total document correctly classified: 98
accuracy: 1.0000


* <b>HW1.6 - Benchmark with Scikit-Learn</b>
- Run the Multinomial Naive Bayes algorithm (using default settings) from SciKit-Learn over the same training data used in HW1.5 and report the Training error (please note some data preparation might be needed to get the Multinomial Naive Bayes algorithm from SkiKit-Learn to run over this dataset)
- Run the Bernoulli Naive Bayes algorithm from SciKit-Learn (using default settings) over the same training data used in HW1.5 and report the Training error 
- Run the Multinomial Naive Bayes algorithm you developed for HW1.5 over the same data used HW1.5 and report the Training error - Please prepare a table to present your results
- Explain/justify any differences in terms of training error rates over the dataset in HW1.5 between your Multinomial Naive Bayes implementation (in Map Reduce) versus the Multinomial Naive Bayes implementation in SciKit-Learn (Hint: smoothing, which we will discuss in next lecture)
- Discuss the performance differences in terms of training error rates over the dataset in HW1.5 between the  Multinomial Naive Bayes implementation in SciKit-Learn with the  Bernoulli Naive Bayes implementation in SciKit-Learn

In [10]:
data = "enronemail_1h.txt"

import string
import numpy as np
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import *

train_data_raw = []
train_labels = []

# load data from file
for line in open(data, 'r'):
    line = line.strip().lower()
    if line == "": continue
    lineArr = line.split('\t')
    if len(lineArr) != 4 or (lineArr[1] != "0" and lineArr[1] != "1"): continue
    spam_flag = lineArr[1]
    subject_body = lineArr[2] + ' ' + lineArr[3]
    subject_body = subject_body.translate(string.maketrans("",""), string.punctuation).strip().lower()
    train_data_raw.append(subject_body)
    train_labels.append(spam_flag)

# fit with count vectorizer
train_data = CountVectorizer().fit_transform(train_data_raw)
#print train_data.shape

# Multinomial NB with default settings:
multiNB_train_accuracy = MultinomialNB().fit(train_data, train_labels).score(train_data, train_labels)
print "MultinomialNB Accuracy: %.4f" % multiNB_train_accuracy

# Bernouli NB with default settings:
bernoulliNB_train_accuracy = BernoulliNB().fit(train_data, train_labels).score(train_data, train_labels)
print "BernoulliNB Accuracy: %.4f" % bernoulliNB_train_accuracy

MultinomialNB Accuracy: 1.0000
BernoulliNB Accuracy: 0.7857


The above code obtains classification accuracy scores for MultinomialNB and BernoulliNB classifier.  The accuracy score for our custom-built MultinomialNB is obtained previously.  Training error is just 1-accuracy.  Thus:

<table>
<tr><th></th><th>Train Accuracy</th><th>Train Error</th></tr>
<tr><th>Sklearn MultinomialNB</th><td>1.00</td><td>0.00</td></tr>
<tr><th>Sklearn BernoulliNB</th><td>0.7857</td><td>0.2143</td></tr>
<tr><th>Custom MultinomialNB</th><td>1.00</td><td>0.00</td></tr>
</table>

* Since we are calculating error rate using training data, it is not surprising that the MultinomialNB from Sklearn and our own custom-built model achieved zero training error.  This means that both models can explain all of the variations in the training data.  However, this does not gaurantee that the model will perform as well on future data.
* BernoulliNB model performs worse because it uses less information from the training data.  The BernoulliNB model only looks at if a term appears in a document, and does not take into account of how many times the term appear in the same document.  This will cause the BernoulliNB model to be unable to distinquish documents with the same words but with different frequencies.